In [1]:
import pandas as pd
import numpy as np

from tabulate import tabulate
import zipfile
import kaggle

import re

In [2]:
!kaggle datasets download -d rashikrahmanpritom/data-science-job-posting-on-glassdoor

data-science-job-posting-on-glassdoor.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
zipfile_name = "data-science-job-posting-on-glassdoor.zip"

with zipfile.ZipFile(zipfile_name,"r") as file:
    file.extract("Uncleaned_DS_jobs.csv")

In [4]:
data = pd.read_csv("Uncleaned_DS_jobs.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              672 non-null    int64  
 1   Job Title          672 non-null    object 
 2   Salary Estimate    672 non-null    object 
 3   Job Description    672 non-null    object 
 4   Rating             672 non-null    float64
 5   Company Name       672 non-null    object 
 6   Location           672 non-null    object 
 7   Headquarters       672 non-null    object 
 8   Size               672 non-null    object 
 9   Founded            672 non-null    int64  
 10  Type of ownership  672 non-null    object 
 11  Industry           672 non-null    object 
 12  Sector             672 non-null    object 
 13  Revenue            672 non-null    object 
 14  Competitors        672 non-null    object 
dtypes: float64(1), int64(2), object(12)
memory usage: 78.9+ KB


In [5]:
data.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [6]:
def check(data):
    dict={}
    for col in data.columns:
        dtype = data[col].dtypes
        valid_instances = data[col].count()
        unique = data[col].nunique()
        null_count = data[col].isnull().sum()
        duplicates = data[col].duplicated().sum()
        dict[col] = dtype,valid_instances,unique,null_count,duplicates
        data_check = pd.DataFrame(dict, index=["dtype","valid_instances","unique","sum_null","duplicates"]).T
    return data_check

check(data)

,dtype,valid_instances,unique,sum_null,duplicates
index,int64,672,672,0,0
Job Title,object,672,172,0,500
Salary Estimate,object,672,30,0,642
Job Description,object,672,489,0,183
Rating,float64,672,32,0,640
Company Name,object,672,432,0,240
Location,object,672,207,0,465
Headquarters,object,672,229,0,443
Size,object,672,9,0,663
Founded,int64,672,103,0,569


In [7]:
def check_unique(data):
    nunique=data.apply(lambda col: col.nunique())
    unique_values = data.apply(lambda col: col.unique())
    data_check = pd.DataFrame({'uni_count': nunique, 'unique_values': unique_values})
    return data_check

check_unique(data)

,uni_count,unique_values
index,672,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
Job Title,172,"[Sr Data Scientist, Data Scientist, Data Scien..."
Salary Estimate,30,"[$137K-$171K (Glassdoor est.), $75K-$131K (Gla..."
Job Description,489,[Description\n\nThe Senior Data Scientist is r...
Rating,32,"[3.1, 4.2, 3.8, 3.5, 2.9, 3.9, 4.4, 3.6, 4.5, ..."
Company Name,432,"[Healthfirst\n3.1, ManTech\n4.2, Analysis Grou..."
Location,207,"[New York, NY, Chantilly, VA, Boston, MA, Newt..."
Headquarters,229,"[New York, NY, Herndon, VA, Boston, MA, Bad Ra..."
Size,9,"[1001 to 5000 employees, 5001 to 10000 employe..."
Founded,103,"[1993, 1968, 1981, 2000, 1998, 2010, 1996, 199..."


In [8]:
pd.reset_option("display.max_colwidth")

In [9]:
import re
pattern = r'\((.*?)\)'
data["Salary Est Ref"]=data["Salary Estimate"].apply(lambda x: re.search(pattern,x).group(1))
data["Salary Estimate"]=data["Salary Estimate"].str.replace(pattern,"",regex=True).str.strip()

In [10]:
data.drop("index",axis=1,inplace=True)

In [11]:
data.columns.to_list()

['Job Title',
 'Salary Estimate',
 'Job Description',
 'Rating',
 'Company Name',
 'Location',
 'Headquarters',
 'Size',
 'Founded',
 'Type of ownership',
 'Industry',
 'Sector',
 'Revenue',
 'Competitors',
 'Salary Est Ref']

In [12]:
col_index=data.columns.get_loc('Salary Est Ref')
data.insert(2,'Salary Est Ref',data.pop('Salary Est Ref'))

In [13]:
data.columns.to_list()

['Job Title',
 'Salary Estimate',
 'Salary Est Ref',
 'Job Description',
 'Rating',
 'Company Name',
 'Location',
 'Headquarters',
 'Size',
 'Founded',
 'Type of ownership',
 'Industry',
 'Sector',
 'Revenue',
 'Competitors']

In [14]:
data.head()

,Job Title,Salary Estimate,Salary Est Ref,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Sr Data Scientist,$137K-$171K,Glassdoor est.,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,Data Scientist,$137K-$171K,Glassdoor est.,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,Data Scientist,$137K-$171K,Glassdoor est.,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,Data Scientist,$137K-$171K,Glassdoor est.,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,Data Scientist,$137K-$171K,Glassdoor est.,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [15]:
min_pattern = r'\$(.*?)K'
max_pattern = r'^\$(.*?)\$(.*?)K'

data['Min Salary ($K)']=data['Salary Estimate'].apply(lambda x: re.search(min_pattern,x).group(1)).astype("int")
data['Max Salary ($K)']=data['Salary Estimate'].apply(lambda x: re.search(max_pattern,x).group(2)).astype("int")



In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Job Title          672 non-null    object 
 1   Salary Estimate    672 non-null    object 
 2   Salary Est Ref     672 non-null    object 
 3   Job Description    672 non-null    object 
 4   Rating             672 non-null    float64
 5   Company Name       672 non-null    object 
 6   Location           672 non-null    object 
 7   Headquarters       672 non-null    object 
 8   Size               672 non-null    object 
 9   Founded            672 non-null    int64  
 10  Type of ownership  672 non-null    object 
 11  Industry           672 non-null    object 
 12  Sector             672 non-null    object 
 13  Revenue            672 non-null    object 
 14  Competitors        672 non-null    object 
 15  Min Salary ($K)    672 non-null    int32  
 16  Max Salary ($K)    672 non

In [17]:
col_index=data.columns.get_loc('Salary Estimate')
for i,col in enumerate(["Min Salary ($K)", "Max Salary ($K)"]):
    data.insert(col_index+i,col,data.pop(col))

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Job Title          672 non-null    object 
 1   Min Salary ($K)    672 non-null    int32  
 2   Max Salary ($K)    672 non-null    int32  
 3   Salary Estimate    672 non-null    object 
 4   Salary Est Ref     672 non-null    object 
 5   Job Description    672 non-null    object 
 6   Rating             672 non-null    float64
 7   Company Name       672 non-null    object 
 8   Location           672 non-null    object 
 9   Headquarters       672 non-null    object 
 10  Size               672 non-null    object 
 11  Founded            672 non-null    int64  
 12  Type of ownership  672 non-null    object 
 13  Industry           672 non-null    object 
 14  Sector             672 non-null    object 
 15  Revenue            672 non-null    object 
 16  Competitors        672 non

In [19]:
data.drop(data.iloc[:,[3]],axis=1,inplace=True)

In [20]:
data.head()

,Job Title,Min Salary ($K),Max Salary ($K),Salary Est Ref,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Sr Data Scientist,137,171,Glassdoor est.,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,Data Scientist,137,171,Glassdoor est.,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,Data Scientist,137,171,Glassdoor est.,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,Data Scientist,137,171,Glassdoor est.,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,Data Scientist,137,171,Glassdoor est.,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [21]:
data["Job Description"][0]

"Description\n\nThe Senior Data Scientist is responsible for defining, building, and improving statistical models to improve business processes and outcomes in one or more healthcare domains such as Clinical, Enrollment, Claims, and Finance. As part of the broader analytics team, Data Scientist will gather and analyze data to solve and address complex business problems and evaluate scenarios to make predictions on future outcomes and work with the business to communicate and support decision-making. This position requires strong analytical skills and experience in analytic methods including multivariate regressions, hierarchical linear models, regression trees, clustering methods and other complex statistical techniques.\n\nDuties & Responsibilities:\n\n• Develops advanced statistical models to predict, quantify or forecast various operational and performance metrics in multiple healthcare domains\n• Investigates, recommends, and initiates acquisition of new data resources from interna

In [22]:
import re

jd_pattern =r"\n\n"

extracted_jd = re.split(jd_pattern,data["Job Description"][0])

extracted_jd

['Description',
 'The Senior Data Scientist is responsible for defining, building, and improving statistical models to improve business processes and outcomes in one or more healthcare domains such as Clinical, Enrollment, Claims, and Finance. As part of the broader analytics team, Data Scientist will gather and analyze data to solve and address complex business problems and evaluate scenarios to make predictions on future outcomes and work with the business to communicate and support decision-making. This position requires strong analytical skills and experience in analytic methods including multivariate regressions, hierarchical linear models, regression trees, clustering methods and other complex statistical techniques.',
 'Duties & Responsibilities:',
 '• Develops advanced statistical models to predict, quantify or forecast various operational and performance metrics in multiple healthcare domains\n• Investigates, recommends, and initiates acquisition of new data resources from int

In [23]:
data['Company Name']

0               Healthfirst\n3.1
1                   ManTech\n4.2
2            Analysis Group\n3.8
3                   INFICON\n3.5
4        Affinity Solutions\n2.9
                 ...            
667                TRANZACT\n3.6
668                         JKGT
669                   AccessHope
670    ChaTeck Incorporated\n5.0
671           1-800-Flowers\n2.7
Name: Company Name, Length: 672, dtype: object

In [24]:
# Remove Excess Number from Company Name
comp_pattern = r'(\n).*$'
data['Company Name']=data['Company Name'].apply(lambda x: re.sub(comp_pattern,"",x))

In [25]:
data.head()

,Job Title,Min Salary ($K),Max Salary ($K),Salary Est Ref,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Sr Data Scientist,137,171,Glassdoor est.,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,Data Scientist,137,171,Glassdoor est.,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,Data Scientist,137,171,Glassdoor est.,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,Data Scientist,137,171,Glassdoor est.,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,Data Scientist,137,171,Glassdoor est.,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [26]:
state_pattern = r'(?<=, )(.*?)$'
state_remove = r', (.*?)$'


data["State"]=data['Location'].apply(lambda x: re.search(state_pattern,x).group(1) if re.search(state_pattern,x) else np.nan)
data["Location"]=data["Location"].apply(lambda x: re.sub(state_remove,'',x) if re.sub(state_remove,'',x) else x)

data.rename({"Location":"City"},axis=1,inplace=True)

In [27]:
col_index = data.columns.get_loc("State")
data.insert(7,data.columns[col_index],data.pop(data.columns[col_index]))

In [28]:
data.head()

,Job Title,Min Salary ($K),Max Salary ($K),Salary Est Ref,Job Description,Rating,Company Name,State,City,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Sr Data Scientist,137,171,Glassdoor est.,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,NY,New York,"New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,Data Scientist,137,171,Glassdoor est.,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,VA,Chantilly,"Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,Data Scientist,137,171,Glassdoor est.,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,MA,Boston,"Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,Data Scientist,137,171,Glassdoor est.,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,MA,Newton,"Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,Data Scientist,137,171,Glassdoor est.,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,NY,New York,"New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [29]:

pattern = r'\$([\d\.]+) (?:to|\+)? \$?([\d\.]+)?'

# Extract minimum and maximum values using regular expressions
min_values = []
max_values = []
for entry in data.Revenue:
    if entry == 'Less than $1 million (USD)':
        min_val = '1'
        max_val = '1'
    elif entry == '-1':
        min_val = '-1'
        max_val = '-1'
    else:
        match = re.search(pattern, entry)
        if match:
            min_val = match.group(1)
            max_val = match.group(2) if match.group(2) else min_val
        else:
            min_val = np.nan
            max_val = np.nan
    min_values.append(min_val)
    max_values.append(max_val)

data[["Min Rev (billion $)","Max Rev (billion $)"]] = pd.DataFrame({'Min': min_values, 'Max': max_values})


In [30]:
col_index=data.columns.get_loc('Competitors')

for i,col in enumerate(["Min Rev (billion $)","Max Rev (billion $)"]):
    data.insert(col_index+i,col,data.pop(col))

In [31]:
data.head()

,Job Title,Min Salary ($K),Max Salary ($K),Salary Est Ref,Job Description,Rating,Company Name,State,City,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Min Rev (billion $),Max Rev (billion $),Competitors
0,Sr Data Scientist,137,171,Glassdoor est.,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,NY,New York,"New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,NaN,NaN,"EmblemHealth, UnitedHealth Group, Aetna"
1,Data Scientist,137,171,Glassdoor est.,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,VA,Chantilly,"Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),1,2,-1
2,Data Scientist,137,171,Glassdoor est.,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,MA,Boston,"Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),100,500,-1
3,Data Scientist,137,171,Glassdoor est.,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,MA,Newton,"Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),100,500,"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,Data Scientist,137,171,Glassdoor est.,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,NY,New York,"New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,NaN,NaN,"Commerce Signals, Cardlytics, Yodlee"


In [32]:
avg_sal_data=(data.iloc[:,1]+data.iloc[:,2])/2

data.insert(3,'Average Salary',avg_sal_data)

In [33]:
for col in data.iloc[:,-3:-1].columns:
    data[col] = data[col].apply(lambda x: float(x))

In [34]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Job Title            672 non-null    object 
 1   Min Salary ($K)      672 non-null    int32  
 2   Max Salary ($K)      672 non-null    int32  
 3   Average Salary       672 non-null    float64
 4   Salary Est Ref       672 non-null    object 
 5   Job Description      672 non-null    object 
 6   Rating               672 non-null    float64
 7   Company Name         672 non-null    object 
 8   State                649 non-null    object 
 9   City                 672 non-null    object 
 10  Headquarters         672 non-null    object 
 11  Size                 672 non-null    object 
 12  Founded              672 non-null    int64  
 13  Type of ownership    672 non-null    object 
 14  Industry             672 non-null    object 
 15  Sector               672 non-null    obj

In [35]:
data.head()

,Job Title,Min Salary ($K),Max Salary ($K),Average Salary,Salary Est Ref,Job Description,Rating,Company Name,State,City,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Min Rev (billion $),Max Rev (billion $),Competitors
0,Sr Data Scientist,137,171,154.0,Glassdoor est.,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,NY,New York,"New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,NaN,NaN,"EmblemHealth, UnitedHealth Group, Aetna"
1,Data Scientist,137,171,154.0,Glassdoor est.,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,VA,Chantilly,"Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),1.0,2.0,-1
2,Data Scientist,137,171,154.0,Glassdoor est.,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,MA,Boston,"Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),100.0,500.0,-1
3,Data Scientist,137,171,154.0,Glassdoor est.,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,MA,Newton,"Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),100.0,500.0,"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,Data Scientist,137,171,154.0,Glassdoor est.,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,NY,New York,"New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,NaN,NaN,"Commerce Signals, Cardlytics, Yodlee"


In [36]:
a=0
for i in ['excel','tableau','power bi','sql','python','aws','big data']:
    data[i] = data["Job Description"].apply(lambda x: 1 if i in x.lower() else 0)
    a+=1
print(f'Total Successful Added Cols: {a}')

Total Successful Added Cols: 7


In [37]:
data['Senior Level'] = data['Job Title'].apply(lambda x: 1 if "senior" in x.lower() else 0)

In [38]:
check(data)

,dtype,valid_instances,unique,sum_null,duplicates
Job Title,object,672,172,0,500
Min Salary ($K),int32,672,26,0,646
Max Salary ($K),int32,672,26,0,646
Average Salary,float64,672,26,0,646
Salary Est Ref,object,672,2,0,670
Job Description,object,672,489,0,183
Rating,float64,672,32,0,640
Company Name,object,672,432,0,240
State,object,649,39,23,632
City,object,672,204,0,468


In [39]:
data.head(10)

,Job Title,Min Salary ($K),Max Salary ($K),Average Salary,Salary Est Ref,Job Description,Rating,Company Name,State,City,...,Max Rev (billion $),Competitors,excel,tableau,power bi,sql,python,aws,big data,Senior Level
0,Sr Data Scientist,137,171,154.0,Glassdoor est.,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,NY,New York,...,NaN,"EmblemHealth, UnitedHealth Group, Aetna",0,0,0,0,0,1,0,0
1,Data Scientist,137,171,154.0,Glassdoor est.,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,VA,Chantilly,...,2.0,-1,0,0,0,1,0,0,1,0
2,Data Scientist,137,171,154.0,Glassdoor est.,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,MA,Boston,...,500.0,-1,1,0,0,0,1,1,0,0
3,Data Scientist,137,171,154.0,Glassdoor est.,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,MA,Newton,...,500.0,"MKS Instruments, Pfeiffer Vacuum, Agilent Tech...",1,0,0,1,1,1,0,0
4,Data Scientist,137,171,154.0,Glassdoor est.,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,NY,New York,...,NaN,"Commerce Signals, Cardlytics, Yodlee",1,0,0,1,1,0,0,0
5,Data Scientist,137,171,154.0,Glassdoor est.,About Us:\n\nHeadquartered in beautiful Santa ...,4.2,HG Insights,CA,Santa Barbara,...,NaN,-1,1,0,0,1,1,0,0,0
6,Data Scientist / Machine Learning Expert,137,171,154.0,Glassdoor est.,Posting Title\nData Scientist / Machine Learni...,3.9,Novartis,MA,Cambridge,...,NaN,-1,0,0,0,0,1,0,0,0
7,Data Scientist,137,171,154.0,Glassdoor est.,Introduction\n\nHave you always wanted to run ...,3.5,iRobot,MA,Bedford,...,2.0,-1,0,0,0,1,1,0,0,0
8,Staff Data Scientist - Analytics,137,171,154.0,Glassdoor est.,Intuit is seeking a Staff Data Scientist to co...,4.4,Intuit - Data,CA,San Diego,...,5.0,"Square, PayPal, H&R Block",0,0,0,0,0,0,0,0
9,Data Scientist,137,171,154.0,Glassdoor est.,Ready to write the best chapter of your career...,3.6,XSELL Technologies,IL,Chicago,...,NaN,-1,0,0,0,1,1,0,0,0


In [40]:
title_sort_data=data["Job Title"].apply(lambda x: "scientist" if "scientist" in x.lower() else ("analyst" if "analyst" in x.lower() else ("engineer" if "engineer" in x.lower() else 'Other')))
data.insert(0,'Title Sorted',title_sort_data)

In [41]:
data[data['Title Sorted']=='engineer']

,Title Sorted,Job Title,Min Salary ($K),Max Salary ($K),Average Salary,Salary Est Ref,Job Description,Rating,Company Name,State,...,Max Rev (billion $),Competitors,excel,tableau,power bi,sql,python,aws,big data,Senior Level
34,engineer,Data Engineer,75,131,103.0,Glassdoor est.,Chef Software is the industry leader in IT aut...,3.6,Chef,NaN,...,100.0,"Puppet, Ansible, SaltStack",1,0,0,1,0,1,1,0
41,engineer,Machine Learning Engineer,75,131,103.0,Glassdoor est.,We deliver our customers peace of mind every d...,3.4,Liberty Mutual Insurance,NH,...,NaN,"Travelers, Allstate, State Farm",1,0,0,0,1,1,1,0
53,engineer,Jr. Data Engineer,75,131,103.0,Glassdoor est.,Jr. Data Engineer\n\nJob Details\nLevel\nEntry...,4.8,IT Concepts,NaN,...,25.0,-1,1,0,0,1,0,0,0,0
55,engineer,Data Analytics Engineer,75,131,103.0,Glassdoor est.,Analytics Engineer-Administration\nSumma Healt...,3.7,Summa Health System,OH,...,500.0,-1,1,1,0,0,0,1,0,0
61,engineer,Machine Learning Engineer,75,131,103.0,Glassdoor est.,The Video & Image Understanding Group develops...,4.5,Systems & Technology Research,MA,...,500.0,-1,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588,engineer,Senior Data Engineer,138,158,148.0,Glassdoor est.,Senior Data Engineer\n\nMaster’s degree in Inf...,2.9,Affinity Solutions,NY,...,NaN,"Commerce Signals, Cardlytics, Yodlee",0,1,0,1,1,0,0,1
590,engineer,Data Engineer,138,158,148.0,Glassdoor est.,Maxar is seeking a Data Engineer to join our t...,3.5,Maxar Technologies,VA,...,5.0,-1,0,0,0,1,1,1,0,0
622,engineer,Data Engineer,87,141,114.0,Glassdoor est.,Alignment Healthcare is a data and technology ...,3.5,Alignment Healthcare,CA,...,NaN,-1,1,1,1,1,0,1,0,0
642,engineer,Data Engineer,92,155,123.5,Glassdoor est.,JOB SUMMARY:\nWe are looking for a savvy Data ...,3.3,Americo Life,MO,...,500.0,-1,1,0,0,1,1,1,0,0


In [42]:
data['State'].isnull().sum()

23

In [43]:
data['State'].unique()

array(['NY', 'VA', 'MA', 'CA', 'IL', 'MO', 'WA', 'DC', nan, 'TN', 'TX',
       'PA', 'AZ', 'WI', 'GA', 'FL', 'NE', 'KS', 'NH', 'NJ', 'LA', 'OH',
       'IN', 'MD', 'CO', 'UT', 'OR', 'MI', 'SC', 'MS', 'AL', 'RI', 'IA',
       'MN', 'OK', 'CT', 'NC', 'DE', 'Anne Arundel, MD', 'WV'],
      dtype=object)

In [44]:
pattern = r'^.*?, '
data['State']=data['State'].apply(lambda x: re.sub(pattern,'',x) if isinstance(x,str) else x)

In [45]:
data['State'].unique()

array(['NY', 'VA', 'MA', 'CA', 'IL', 'MO', 'WA', 'DC', nan, 'TN', 'TX',
       'PA', 'AZ', 'WI', 'GA', 'FL', 'NE', 'KS', 'NH', 'NJ', 'LA', 'OH',
       'IN', 'MD', 'CO', 'UT', 'OR', 'MI', 'SC', 'MS', 'AL', 'RI', 'IA',
       'MN', 'OK', 'CT', 'NC', 'DE', 'WV'], dtype=object)

In [46]:
data["HQ Same State"] = data.apply(lambda row: 1 if isinstance(row['State'], str) and row['State'] in row['Headquarters'] else 0, axis=1)


In [47]:
data.head()

,Title Sorted,Job Title,Min Salary ($K),Max Salary ($K),Average Salary,Salary Est Ref,Job Description,Rating,Company Name,State,...,Competitors,excel,tableau,power bi,sql,python,aws,big data,Senior Level,HQ Same State
0,scientist,Sr Data Scientist,137,171,154.0,Glassdoor est.,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,NY,...,"EmblemHealth, UnitedHealth Group, Aetna",0,0,0,0,0,1,0,0,1
1,scientist,Data Scientist,137,171,154.0,Glassdoor est.,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,VA,...,-1,0,0,0,1,0,0,1,0,1
2,scientist,Data Scientist,137,171,154.0,Glassdoor est.,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,MA,...,-1,1,0,0,0,1,1,0,0,1
3,scientist,Data Scientist,137,171,154.0,Glassdoor est.,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,MA,...,"MKS Instruments, Pfeiffer Vacuum, Agilent Tech...",1,0,0,1,1,1,0,0,0
4,scientist,Data Scientist,137,171,154.0,Glassdoor est.,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,NY,...,"Commerce Signals, Cardlytics, Yodlee",1,0,0,1,1,0,0,0,1


In [48]:
data.drop('Competitors',axis=1,inplace=True)

In [49]:
data.to_csv('Cleaned_DS_jobs.csv',sep=',')

# SQL QUERY WITH SQLITE

In [50]:
import sqlite3

In [51]:
cnn = sqlite3.connect('glassdoor_sql.db')

In [52]:
%load_ext sql
%sql sqlite:///glassdoor_sql.db

In [53]:
%%sql

DROP TABLE IF EXISTS glassdoor

 * sqlite:///glassdoor_sql.db
Done.


[]

In [54]:
data.to_sql('glassdoor',cnn)

672

In [55]:
%%sql

select "Title Sorted", Count(*) as total_jobs from glassdoor
    GROUP BY "Title Sorted"
    Order By total_jobs DESC

 * sqlite:///glassdoor_sql.db
Done.


Title Sorted,total_jobs
scientist,519
engineer,78
analyst,55
Other,20


In [56]:
%%sql

select "Title Sorted", AVG("Rating") as average_rating from glassdoor
    GROUP BY "Title Sorted"
    ORDER BY average_rating DESC

 * sqlite:///glassdoor_sql.db
Done.


Title Sorted,average_rating
engineer,3.9615384615384617
Other,3.8200000000000003
analyst,3.7636363636363637
scientist,3.414450867052023


In [57]:
data.columns

Index(['Title Sorted', 'Job Title', 'Min Salary ($K)', 'Max Salary ($K)',
       'Average Salary', 'Salary Est Ref', 'Job Description', 'Rating',
       'Company Name', 'State', 'City', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue',
       'Min Rev (billion $)', 'Max Rev (billion $)', 'excel', 'tableau',
       'power bi', 'sql', 'python', 'aws', 'big data', 'Senior Level',
       'HQ Same State'],
      dtype='object')

In [58]:
%%sql
With analyst_cte(rating,salary) AS
(SELECT "Rating", "Average Salary" FROM glassdoor WHERE "Title Sorted" = "analyst")

SELECT "Job Title", "Min Salary ($K)", "Max Salary ($K)",
       "Average Salary", "Salary Est Ref", "Rating", "Company Name", "State",
       "City", "HQ Same State", "Size", "Founded", "Type of ownership",
       "Industry", "Sector", "Revenue", "excel", "tableau", "power bi", "sql",
       "python"
    FROM glassdoor
WHERE "Title Sorted" = "analyst"
AND "Rating" > (SELECT AVG(rating) FROM analyst_cte)
AND "Average Salary" > (SELECT AVG(salary) FROM analyst_cte)
AND "Senior Level" = 0
AND "aws" = 0
AND "big data" = 0
ORDER BY "Average Salary" DESC, "Rating" DESC
LIMIT 10;

 * sqlite:///glassdoor_sql.db
Done.


Job Title,Min Salary ($K),Max Salary ($K),Average Salary,Salary Est Ref,Rating,Company Name,State,City,HQ Same State,Size,Founded,Type of ownership,Industry,Sector,Revenue,excel,tableau,power bi,sql,python
Analytics - Business Assurance Data Analyst,145,225,185.0,Employer est.,4.6,GreatAmerica Financial Services,IA,Cedar Rapids,1,501 to 1000 employees,1992,Company - Private,Lending,Finance,$100 to $500 million (USD),1,1,0,1,0
"Health Plan Data Analyst, Sr",145,225,185.0,Employer est.,3.8,Cook Children's Health Care System,TX,Fort Worth,1,5001 to 10000 employees,1918,Nonprofit Organization,Health Care Services & Hospitals,Health Care,$100 to $500 million (USD),0,0,0,0,0
Data Analyst,141,225,183.0,Glassdoor est.,4.5,ShorePoint,VA,Reston,0,1 to 50 employees,2017,Company - Private,IT Services,Information Technology,Unknown / Non-Applicable,1,1,1,1,0
Clinical Data Analyst,128,201,164.5,Glassdoor est.,3.8,NYSTEC,NY,New York,1,51 to 200 employees,1996,Nonprofit Organization,Consulting,Business Services,$25 to $50 million (USD),0,0,0,1,1
Analytics - Business Assurance Data Analyst,124,198,161.0,Glassdoor est.,4.6,GreatAmerica Financial Services,IA,Cedar Rapids,1,501 to 1000 employees,1992,Company - Private,Lending,Finance,$100 to $500 million (USD),1,1,0,1,0
Data Analyst II,137,171,154.0,Glassdoor est.,4.2,"Insight Enterprises, Inc.",TX,Plano,0,5001 to 10000 employees,1988,Company - Public,Enterprise Software & Network Solutions,Information Technology,$5 to $10 billion (USD),1,0,0,0,0
Enterprise Data Analyst (Enterprise Portfolio Management Office),138,158,148.0,Glassdoor est.,3.9,Navy Federal Credit Union,VA,Merrifield,1,10000+ employees,1933,Other Organization,Banks & Credit Unions,Finance,Unknown / Non-Applicable,0,1,1,1,0
Data Analyst,122,146,134.0,Glassdoor est.,3.9,Nolij Consulting,VA,Falls Church,1,51 to 200 employees,2012,Company - Public,Consulting,Business Services,$1 to $5 million (USD),1,1,0,1,1
